# Планы на будущее

## Данные

1. Рассматривается возможность использовать данные срезов одного керна вместо множества случайных;
2. Более высокое разрешение изначальных изображений (условные 1024x1024 вместо 500x500);
3. (В идеале) Настоящие данные заказчика;

## Предобработка
1. Исследования влияния `GaussianBlur` на качество;
2. Добавление новых аугментаций и преобразований;

## Модели

1. Создать Diffusion Model:
   * DDNM
2. Создать GAN (в приоретете):
   * ESRGAN (в приоретете)
   * CycleGAN
3. Найти и создать более продвинутую CNN, чем U-Net;
4. Эксперименты с параметрами U-Net;
5. 

## Процесс обучения

1. Добавить `scheduler`;
2. Эксперименты с разными `optimizer`;
3. Исследования влияния `BatchNorm` на качество:
4. Исследования в использовании `L1`, `MSE`, `L1 + (1 - SSIM)` как функций потерь; 

## Код

1. Сделать код более переиспользуемым:
   * вынести этапы предобработки (и, возможно, загрузчики) в отдельный модуль;
2. Сделать предобработку общей для всех моделей, где это возможно;

## Логирование

1. Добавить поддержку W&B или MLflow;
2. Создать GitHub-репозиторий.

### Что делать в первую очередь (высокий приоритет)

Диапазоны, метрики и протокол

Тренируй и валидируй в [0,1].

Для PSNR/SSIM делай shave = scale (обрезай по краям) — иначе апсемплинговые границы портят метрику.

Стабильный SSIM: fp32, kernel_size=7 (или 5 на мелких патчах), gaussian_weights=True, eps > 0.

Если используешь Normalize(mean,std) для входа — для метрик обратно денормализуй.

Патчи и аугментации (дают больше, чем выбор лосса)

Подбери patch_size: для x2 попробуй 128→192→256. Б óльший патч = больше контекста → обычно ↑SSIM/PSNR.

Аугментации умеренные: flips/rot90, лёгкий GaussianNoise. Откажись от сильного blur — он конфликтует с задачей SR (LR уже деградированы).

Следи, чтобы HR/LR кропы были строго согласованы по координатам.

Нормализации в U-Net

Убрать BatchNorm — хорошая идея для маленьких батчей. Альтернатива: GroupNorm(32) или InstanceNorm — часто стабильнее.

Если BN оставляешь, валидируй с model.eval() и не смешивай разные масштабы в одном батче.

Оптимизация и стабильность шага

AdamW lr=2e-4, weight_decay=1e-4 (или Adam без WD, если лезут артефакты).

CosineAnnealing с warmup 3–5 эпох или OneCycleLR — на SR обычно надёжно.

Градиент-клиппинг: clip_grad_norm_(..., 1.0).

EMA весов (decay ~0.999) часто даёт +0.05–0.15 dB PSNR.

Лосс — стабильная база

Вместо L1 + (1-SSIM) попробуй: Charbonnier (√(x²+ε²)) + MS-SSIM.
Рецепт: 0.84 * Charbonnier + 0.16 * (1 - MS-SSIM); MS-SSIM считать в fp32, затем смешивать.

Если остаёшься на L1+SSIM: считай SSIM в fp32 и клампи ssim.clamp(0,1).

### Дальше по важности (средний приоритет)

Глубина/ширина U-Net

Сделай глубину 4–5 даунсемплов, базовые каналы 64 (или 48, если память).

В декодере избегай “шахматки”: upsample (nearest/bilinear) + conv вместо ConvTranspose2d. Это уменьшит артефакты и даст +к стабильности.

Резидуализация блока

Перейди на Res-UNet (residual conv-блоки внутри энкодера/декодера). Часто даёт заметный прирост при той же сложности.

Внимание (легковесно)

SE/SCSE/CBAM в скип-конкатенациях или в бутылочном блоке. Дёшево и иногда ощутимо ↑SSIM на текстурах.

Куррикулум по масштабам

Обучи x2 до сходимости → инициализируй x4 этими весами → финтюн с lr в 3–5 раз меньше. Обычно лучше, чем “параллельно”.

Валид. протокол и сиды

Фиксируй сиды, список валид-патчей/картинок, единый shave, единые пред-/пост-процедуры. Иначе шум в +0.01 SSIM легко “съест” эффект.

### Низкий приоритет / точечные усилители

Перцептуальный лосс

VGG-перцептуальный (relu2_2/relu3_3) с маленьким весом (например 0.01–0.05) — иногда улучшает визуально, но часто снижает PSNR. Пробуй, если важна визуалка.

TTA / self-ensemble на тесте

x8-self-ensemble (flip/rot) даёт +0.05–0.2 dB без ретрейна.

Регуляризация

TV-регуляризатор на предсказании с маленьким весом может убирать “дрожь” на гранях. Аккуратно, чтобы не замылить.

Базовый “эталон” конфиг, от которого плясать

Data: patch=192 (x2), flips+rot90, лёгкий noise; batch_size=8; workers=8.

Model: UNet(depth=5, base_ch=64), без BN (или GroupNorm), upsample+conv, LeakyReLU.

Loss: 0.84 * Charbonnier(eps=1e-3) + 0.16 * (1 - MS-SSIM); MS-SSIM в fp32.

Optim: AdamW(lr=2e-4, wd=1e-4), cosine + 5 эпох warmup, grad-clip=1.0, EMA=0.999.

Training: AMP включён, кроме SSIM; epochs_x2=150 с ранней остановкой (patience=20, min_delta=0.002 SSIM).

Validation: model.eval(), no_grad, денормализация перед метриками, PSNR/SSIM со shave=scale.

Logging: логи лосса по компонентам (L1/Charbonnier и (1-SSIM)) и финального суммарного; следи за их соотношением.

Что проверить в твоих текущих запусках (быстрые выигрыши)

Убедись, что SSIM считается в fp32 и с корректным data_range.

Попробуй patch_size=192/256 — часто даёт больший прирост, чем замена лосса.

Оставь вариант без нормализаций (или с GroupNorm) — на маленьком батче это часто лучший сетап.

Включи EMA и grad-clip — добавляет стабильности и пару сотых SSIM “на халяву”.

Проверь shave при валидации — без него сравнение разных конфигов шумное.